In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np

df = pd.read_parquet('./data')
print(f'total dataset: {len(df)} rows')

## filters
# df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
df = df[df['stabilized_spread'] > 0.3]  # nem tul kis spread

for col in df.columns:
    if 'nr_trades' in col or 'price_delta' in col and not col.startswith('last'):
        df.drop(col, axis=1, inplace=True)
    if 'nr_trades' in col:
        df.drop(col, axis=1, inplace=True)
    if 'past' in col:
        df.drop(col, axis=1, inplace=True)
    if '_spread' == col[1:]:
        df.drop(col, axis=1, inplace=True)
    pass

df_min = df[df.wave_direction == 'min'].copy()
df_max = df[df.wave_direction == 'max'].copy()

df_min.drop('wave_direction', axis=1, inplace=True)
df_max.drop('wave_direction', axis=1, inplace=True)
df = None
print(f'min dataset: {len(df_min)}')
print(f'max dataset: {len(df_max)}')

df_min.sort_index(axis=1, inplace=True)
df_max.sort_index(axis=1, inplace=True)

display(df_max)

df_max.last_price_delta_since_stabilized = df_max.last_price_delta_since_stabilized * -1
df_max.last_price_delta_since_stabilized.describe()

total dataset: 134367 rows
min dataset: 47193
max dataset: 47671


,0_amount_mean,0_nr_trades,0_price_delta,1_amount_mean,1_nr_trades,1_price_delta,2_amount_mean,2_nr_trades,2_price_delta,3_amount_mean,...,3_price_delta,4_amount_mean,4_nr_trades,4_price_delta,last_price_delta_since_stabilized,stabilized_amount_mean,stabilized_at_ms,stabilized_gasp,stabilized_nr_trades,stabilized_spread
index,,,,,,,,,,,,,,,,,,,,,
0,0.0305,28.0,0.12,0.0815,30.0,0.11,0.0792,31.0,0.11,0.0767,...,0.11,0.0725,35.0,0.01,-0.78,0.0706,108,-0.4798,36.0,0.91
0,0.0078,5.0,0.84,0.0072,6.0,0.84,0.0122,8.0,0.84,0.0173,...,0.11,0.0301,41.0,0.00,-1.84,0.0295,186,0.2329,42.0,1.67
0,0.0348,30.0,0.14,0.0531,32.0,0.06,0.0503,34.0,0.00,0.0489,...,0.00,0.0481,36.0,0.00,-0.74,0.0469,151,-0.8172,37.0,1.83
0,0.0031,4.0,1.11,0.0027,5.0,1.11,0.0036,7.0,1.09,0.0058,...,0.77,0.0188,25.0,0.54,-0.89,0.0167,170,-0.9874,38.0,1.81
0,0.0318,92.0,0.69,0.0315,93.0,0.69,0.0317,95.0,0.69,0.0341,...,0.69,0.0360,145.0,0.00,0.00,0.0345,822,-0.5774,152.0,3.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0017,3.0,0.41,0.0108,5.0,0.41,0.0095,6.0,0.02,0.0084,...,0.00,0.0074,8.0,0.00,0.00,0.0067,725,-0.1821,9.0,0.63
0,0.0066,4.0,0.17,0.0086,6.0,0.15,0.0113,8.0,0.00,0.0155,...,0.00,0.0141,10.0,0.00,-1.33,0.0131,432,-0.0158,11.0,0.45
0,0.0054,3.0,0.52,0.0042,4.0,0.52,0.0031,6.0,0.43,0.0035,...,0.00,0.0037,9.0,0.00,-0.38,0.0036,427,-0.1673,10.0,0.64


count    47671.000000
mean         0.540317
std          0.950748
min         -0.000000
25%         -0.000000
50%          0.180000
75%          0.670000
max         17.160000
Name: last_price_delta_since_stabilized, dtype: float64

In [69]:
a = df_min.corr().last_price_delta_since_stabilized * 1000
display(a.sort_values(axis=0, ascending=False))

a = df_max.corr().last_price_delta_since_stabilized * 1000
display(a.sort_values(axis=0, ascending=False))


last_price_delta_since_stabilized    1000.000000
stabilized_spread                     123.918287
4_price_delta                          88.350255
stabilized_amount_mean                 70.177820
0_price_delta                          65.923447
1_price_delta                          63.136621
4_amount_mean                          60.323552
2_price_delta                          59.494922
3_amount_mean                          58.861296
3_price_delta                          56.969403
2_amount_mean                          54.322352
1_amount_mean                          53.223830
0_amount_mean                          50.861739
stabilized_gasp                        11.784142
0_nr_trades                           -33.135432
1_nr_trades                           -37.499456
2_nr_trades                           -43.249126
3_nr_trades                           -49.149722
stabilized_nr_trades                  -51.989173
4_nr_trades                           -59.978086
stabilized_at_ms    

last_price_delta_since_stabilized    1000.000000
stabilized_spread                     122.643480
4_price_delta                          90.993986
stabilized_amount_mean                 69.117536
3_price_delta                          61.301452
4_amount_mean                          55.909274
2_price_delta                          54.266697
0_price_delta                          54.241868
3_amount_mean                          52.577247
1_price_delta                          48.174192
1_amount_mean                          47.040900
2_amount_mean                          46.378958
0_amount_mean                          44.175660
stabilized_gasp                        -4.394740
0_nr_trades                           -23.902719
1_nr_trades                           -28.209372
2_nr_trades                           -35.360784
3_nr_trades                           -41.295565
stabilized_nr_trades                  -41.358331
4_nr_trades                           -51.975494
stabilized_at_ms    

In [82]:
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler


def train(df, loss_function):
    # model = CatBoostRegressor(learning_rate=0.01, depth=7, loss_function=loss_function, random_state=0, verbose=False, iterations=1500)
    model = CatBoostRegressor(learning_rate=0.15, depth=6, loss_function=loss_function, random_state=0, verbose=False)
    pipeline = Pipeline(steps=[
        ('preprocessor', FeatureEngineer()),
        ('model', model)
    ])

    df = df.copy()
    y = df['last_price_delta_since_stabilized']
    X = df.drop('last_price_delta_since_stabilized', axis=1)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=0)

    pipeline.fit(X_train, y_train)
    score = pipeline.score(X_valid, y_valid)
    display(score)
    return model, score


# min_model, min_score = train(df_min, loss_function='MAPE')
# max_model, max_score = train(df_max, loss_function='MAPE')

# min_model, min_score = train(df_min, loss_function='RMSE')
# max_model, max_score = train(df_max, loss_function='RMSE')

# min_model, min_score = train(df_min, loss_function='Poisson') # 0.12
# max_model, max_score = train(df_max, loss_function='Poisson') # 0.14

# min_model, min_score = train(df_min, loss_function='Huber:delta=0.4')
# max_model, max_score = train(df_max, loss_function='Huber:delta=0.4')

# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.8')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.7')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.6')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.5')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.4')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.3')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.2')

max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.7')



0.15643907191202266

In [83]:
from spreadsurfer import now_isoformat

save = True
if save:
    # min_model.save_model(f'./models/{now_isoformat()}-min_-quantile0.7-score-{1000 * round(min_score, 3)}.cat')
    max_model.save_model(f'./models/{now_isoformat()}-max_quantile0.7-score-{1000 * round(max_score, 3)}.cat')
